# Topological Evolution During Network Assembly

This notebook demonstrates the use of topological data analysis (TDA)
to characterize coordination network assembly. We explore:

1. Persistent homology computation
2. Betti number evolution
3. Rigidity analysis
4. How topology relates to emergent properties

In [ ]:
import sys
sys.path.insert(0, '../..')

import numpy as np
import matplotlib.pyplot as plt

from assembly_net.data.synthetic_assembly_simulator import (
    CoordinationNetworkSimulator,
    SimulationParameters,
)
from assembly_net.topology.persistent_homology import (
    PersistentHomologyComputer,
    compute_betti_numbers,
)
from assembly_net.topology.graph_rigidity import (
    RigidityAnalyzer,
    compute_bulk_modulus,
)
from assembly_net.topology.descriptors import (
    TopologicalDescriptorExtractor,
    compute_percolation_threshold,
    compute_loop_formation_rate,
)
from assembly_net.utils.visualization import (
    plot_persistence_diagram,
    plot_betti_evolution,
)

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Generate Assembly Trajectory

In [ ]:
params = SimulationParameters(
    num_metal_ions=30,
    num_ligands=60,
    metal_valency=6,
    ligand_valency=2,
    ph=7.0,
    total_time=150.0,
    snapshot_interval=1.0,
    seed=42,
)

simulator = CoordinationNetworkSimulator(params)
trajectory = simulator.run()

print(f"Trajectory: {len(trajectory.states)} states, duration {trajectory.duration:.1f}")

## 2. Betti Number Evolution

Track the evolution of topological invariants:
- **β₀**: Number of connected components (clusters)
- **β₁**: Number of independent cycles (loops)

In [ ]:
fig = plot_betti_evolution(trajectory)
plt.show()

In [ ]:
# Detailed evolution analysis
times = []
betti_0 = []
betti_1 = []
num_edges = []

for state in trajectory.states:
    times.append(state.time)
    betti = compute_betti_numbers(state.graph)
    betti_0.append(betti[0])
    betti_1.append(betti[1])
    num_edges.append(len(state.graph.edges))

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# β₀ vs time
axes[0, 0].plot(times, betti_0, 'r-', linewidth=2)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('β₀ (Components)')
axes[0, 0].set_title('Connected Components')

# β₁ vs time
axes[0, 1].plot(times, betti_1, 'b-', linewidth=2)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('β₁ (Cycles)')
axes[0, 1].set_title('Independent Cycles')

# β₁/β₀ ratio
ratio = [b1 / max(b0, 1) for b0, b1 in zip(betti_0, betti_1)]
axes[1, 0].plot(times, ratio, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('β₁/β₀')
axes[1, 0].set_title('Loop-to-Component Ratio')

# β₁ vs edges
axes[1, 1].scatter(num_edges, betti_1, c=times, cmap='viridis', alpha=0.7)
axes[1, 1].set_xlabel('Number of Edges')
axes[1, 1].set_ylabel('β₁ (Cycles)')
axes[1, 1].set_title('Cycles vs Network Size')
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Time')

plt.tight_layout()
plt.show()

## 3. Persistent Homology

Compute persistence diagrams for the final network structure.

In [ ]:
ph_computer = PersistentHomologyComputer(max_dimension=1)
final_features = ph_computer.compute(trajectory.final_state.graph)

print("Topological Features of Final Network:")
print(f"  Betti numbers: {final_features.betti_numbers}")
print(f"  Persistence entropy: {final_features.persistence_entropy:.4f}")
print(f"  Total persistence: {final_features.total_persistence:.4f}")

In [ ]:
fig = plot_persistence_diagram(final_features.persistence_diagram)
plt.show()

## 4. Rigidity Analysis

Analyze mechanical rigidity using Maxwell counting and pebble game.

In [ ]:
rigidity_analyzer = RigidityAnalyzer(dimension=3)

# Track rigidity metrics over time
maxwell_counts = []
mean_coords = []
is_rigid = []

for state in trajectory.states:
    metrics = rigidity_analyzer.analyze(state.graph)
    maxwell_counts.append(metrics.maxwell_count)
    mean_coords.append(metrics.mean_coordination)
    is_rigid.append(metrics.is_rigid)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Maxwell count
axes[0].plot(times, maxwell_counts, 'b-', linewidth=2)
axes[0].axhline(y=0, color='r', linestyle='--', label='Rigidity threshold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Maxwell Count')
axes[0].set_title('Maxwell Counting')
axes[0].legend()

# Mean coordination
axes[1].plot(times, mean_coords, 'g-', linewidth=2)
axes[1].axhline(y=6.0, color='r', linestyle='--', label='z_c = 2d = 6')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Mean Coordination <z>')
axes[1].set_title('Coordination Number')
axes[1].legend()

# Rigidity indicator
axes[2].fill_between(times, is_rigid, alpha=0.5)
axes[2].set_xlabel('Time')
axes[2].set_ylabel('Is Rigid')
axes[2].set_title('Rigidity Status')
axes[2].set_ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

## 5. Percolation Threshold

In [ ]:
perc_time, final_strength = compute_percolation_threshold(trajectory)

print(f"Percolation time: {perc_time:.1f}" if perc_time else "Did not percolate")
print(f"Final percolation strength: {final_strength:.3f}")

In [ ]:
# Plot largest component fraction
trajectory.compute_all_properties()

lcc_fraction = [
    (s.largest_component_size or 0) / max(1, s.graph.num_nodes)
    for s in trajectory.states
]

plt.figure(figsize=(10, 6))
plt.plot(times, lcc_fraction, 'b-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', label='Percolation threshold')
if perc_time:
    plt.axvline(x=perc_time, color='g', linestyle=':', label=f'Percolation at t={perc_time:.1f}')
plt.xlabel('Time')
plt.ylabel('Largest Component Fraction')
plt.title('Percolation Transition')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Full Topological Descriptor Extraction

In [ ]:
extractor = TopologicalDescriptorExtractor(
    compute_homology=True,
    compute_rigidity=True,
)

# Extract for final state
final_descriptor = extractor.extract(trajectory.final_state.graph)

print("Full Topological Descriptor:")
print(f"  Nodes: {final_descriptor.num_nodes}")
print(f"  Edges: {final_descriptor.num_edges}")
print(f"  Components: {final_descriptor.num_components}")
print(f"  Triangles: {final_descriptor.num_triangles}")
print(f"  Loop density: {final_descriptor.loop_density:.4f}")
print(f"  Clustering coefficient: {final_descriptor.clustering_coefficient:.4f}")
print(f"  Percolated: {final_descriptor.is_percolated}")
print(f"  Bulk modulus: {final_descriptor.bulk_modulus:.4f}")
print(f"  Shear modulus: {final_descriptor.shear_modulus:.4f}")

## 7. Topology-Property Correlation

How do topological features correlate with emergent properties?

In [ ]:
from assembly_net.data.synthetic_assembly_simulator import generate_dataset

# Generate dataset
trajectories = generate_dataset(num_samples=100, seed=42)

# Extract features and labels
loop_densities = []
mechanical_scores = []
percolated = []

for traj in trajectories:
    desc = extractor.extract(traj.final_state.graph)
    loop_densities.append(desc.loop_density)
    mechanical_scores.append(traj.labels['mechanical_score'])
    percolated.append(traj.labels['percolation_achieved'])

# Plot correlation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

colors = ['red' if p else 'blue' for p in percolated]
axes[0].scatter(loop_densities, mechanical_scores, c=colors, alpha=0.6)
axes[0].set_xlabel('Loop Density')
axes[0].set_ylabel('Mechanical Score')
axes[0].set_title('Loop Density vs Mechanical Score')

# Correlation coefficient
corr = np.corrcoef(loop_densities, mechanical_scores)[0, 1]
axes[0].annotate(f'r = {corr:.3f}', xy=(0.05, 0.95), xycoords='axes fraction')

# Box plot by percolation status
scores_percolated = [s for s, p in zip(mechanical_scores, percolated) if p]
scores_not_percolated = [s for s, p in zip(mechanical_scores, percolated) if not p]

axes[1].boxplot([scores_not_percolated, scores_percolated], labels=['Not Percolated', 'Percolated'])
axes[1].set_ylabel('Mechanical Score')
axes[1].set_title('Mechanical Score by Percolation Status')

plt.tight_layout()
plt.show()